In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import psutil
import glob
import os

Using TensorFlow backend.


# defining global variabals

In [2]:
# To prevent system form flooding the RAM
FREE_RAM_LIMIT = 2_000_000_000

# Getting the availible RAM
*_, FREE_RAM = psutil.virtual_memory()
FREE_RAM -= FREE_RAM_LIMIT
FREE_RAM //= 1024

# All images are 28 * 28 * 1 = 784
# 1 because images are grey scalled images
IMAGE_ROWS = 28
IMAGE_COLS = 28
IMAGE_SIZE = 784

# Path to data
IMAGE_DIR = "./data/"

# Total number of numpy file
NUM_IMAGES = len(glob.glob1(IMAGE_DIR, "*.npy"))

# How many element in each image we have to take so we don't run out of memory
TOTAL_ELEMENTS_PER_IMAGE = 2000  # FREE_RAM // (IMAGE_SIZE * NUM_IMAGES)

# setting for all labels a unique value
label_map = {}

for i, file in enumerate(os.listdir(IMAGE_DIR)):
    label_map[file[:-4]] = i

# Loading the data

In [3]:
x_train = np.ndarray(shape=(0, 784), dtype=np.float32)
y_train = np.array([], dtype=np.uint16)

for file in os.listdir(IMAGE_DIR):
    print(file)
    elements = np.load(IMAGE_DIR + file)[:TOTAL_ELEMENTS_PER_IMAGE] / 255
    x_train = np.concatenate((x_train, elements))
    labels = np.full(elements.shape[0], label_map[file[:-4]])
    y_train = np.append(y_train, labels)

x_train, x_validate, y_train, y_validate = tts(x_train, y_train, test_size=.2)

aircraft_carrier.npy
airplane.npy
alarm_clock.npy
ambulance.npy
angel.npy
animal_migration.npy
ant.npy
anvil.npy
apple.npy
arm.npy
asparagus.npy
axe.npy
backpack.npy
banana.npy
bandage.npy
barn.npy
baseball.npy
baseball_bat.npy
basket.npy
basketball.npy
bat.npy
bathtub.npy
beach.npy
bear.npy
beard.npy
bed.npy
bee.npy
belt.npy
bench.npy
bicycle.npy
binoculars.npy
bird.npy
birthday_cake.npy
blackberry.npy
blueberry.npy
book.npy
boomerang.npy
bottlecap.npy
bowtie.npy
bracelet.npy
brain.npy
bread.npy
bridge.npy
broccoli.npy
broom.npy
bucket.npy
bulldozer.npy
bus.npy
bush.npy
butterfly.npy
cactus.npy
cake.npy
calculator.npy
calendar.npy
camel.npy
camera.npy
camouflage.npy
campfire.npy
candle.npy
cannon.npy
canoe.npy
car.npy
carrot.npy
castle.npy
cat.npy
ceiling_fan.npy
cello.npy
cell_phone.npy
chair.npy
chandelier.npy
church.npy
circle.npy
clarinet.npy
clock.npy
cloud.npy
coffee_cup.npy
compass.npy
computer.npy
cookie.npy
cooler.npy
couch.npy
cow.npy
crab.npy
crayon.npy
crocodile.npy
crown.

# Reshape the data

In [4]:
BATCH_SIZE = 512
IMAGE_SHAPE = (IMAGE_ROWS, IMAGE_COLS, 1)

x_train = x_train.reshape(x_train.shape[0], *IMAGE_SHAPE)
x_validate = x_validate.reshape(x_validate.shape[0], *IMAGE_SHAPE)


# Creating the Model
## Define the model

In [5]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3,
                 activation="relu", input_shape=IMAGE_SHAPE))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.3))
model.add(Conv2D(filters=256, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.35))
model.add(Conv2D(filters=512, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(600, activation="relu"))
model.add(Dense(450, activation="relu"))
model.add(Dense(NUM_IMAGES, activation="softmax"))

print(model.sumary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
"""tensorboard = TensorBoard(log_dir=r'logs\{}'.format('cnn_layer'),
                         write_graph=True,
                         write_grads=True,
                         histogram_freq=1,
                         write_images=True)"""

model.compile(loss="sparse_categorical_crossentropy",
             optimizer=Adam(lr=0.001),
             metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1024, verbose=1,
          validation_data=(x_validate, y_validate))

Train on 552000 samples, validate on 138000 samples
Epoch 1/50
552000/552000 [==============================] - 113s 205us/step - loss: 2.0795 - acc: 0.5163 - val_loss: 1.9947 - val_acc: 0.5359
Epoch 2/50
349696/552000 [==================>...........] - ETA: 38s - loss: 1.8705 - acc: 0.5578

KeyboardInterrupt: 

In [ ]:
model.save('keras.h5')